<h1 style="text-align:center;">ESCUELA POLITÉCNICA NACIONAL</h1>
<h2 style="text-align:center;">FACULTAD DE INGENIERÍA DE SISTEMAS</h2>
<h3 style="text-align:center;">INGENIERÍA DE COMPUTACIÓN</h3>

**PERÍODO ACADÉMICO:** 2025-B

**ASIGNATURA:** ICCD412 Métodos Numéricos  

**GRUPO:** GR2  

**TIPO DE INSTRUMENTO:** Proyecto sobre calidad de aire en Quito 

**ALUMNOS:** Guerra Melissa, Velastegui Joel

## LIBRERÍAS IMPLEMENTADAS

In [8]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# IMPLEMENTACIÓN

# DEFINICIÓN DE FUNCIONES
## PREPROCESAMIENTO
Son funciones encargadas de la carga, limpieza e imputación de datos. Se aplica un criterio físico para la Radiación Solar (RS), limitando la interpolación a las horas diurnas.

In [9]:
def procesar_dataset_aire(
    ruta_archivo: str,
    hora_inicio_sol: int = 6,
    hora_fin_sol: int = 18,
    variables_interes: list = None,
    nombre_salida: str = "dataset_procesado"
) -> pd.DataFrame:
    """
    Carga y procesa datos de calidad del aire, aplicando interpolación lineal.
    """
    # 1. Lectura y configuración del índice temporal
    df = pd.read_excel(ruta_archivo)
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df.sort_values("Fecha").set_index("Fecha")

    # 2. Selección de variables
    if variables_interes is None:
        variables_interes = ["CO", "NO2", "PM2.5", "VEL", "TMP", "RS"]
    
    # 3. Separación de variables: Continuas vs. Cíclicas (Radiación)
    vars_continuas = [v for v in variables_interes if v != "RS"]
    
    # Imputación lineal para variables continuas
    if vars_continuas:
        df[vars_continuas] = df[vars_continuas].interpolate(
            method="time", 
            limit_direction="both"
        )

    # 4. Tratamiento específico para Radiación Solar (RS)
    if "RS" in variables_interes:
        horas = df.index.hour
        mask_dia = (horas >= hora_inicio_sol) & (horas <= hora_fin_sol)
        
        # Interpolamos solo de día
        df.loc[mask_dia, "RS"] = df.loc[mask_dia, "RS"].interpolate(
            method="time", limit_direction="both"
        )
        # De noche asumimos 0 o mantenemos nulos tratados
        df["RS"] = df["RS"].fillna(0)

    # 5. Limpieza final y exportación
    df_final = df.dropna(subset=variables_interes)
    df_final.to_csv(f"{nombre_salida}.csv")
    print(f"Dataset procesado guardado como: {nombre_salida}.csv")

    return df_final

In [10]:
# Ejecución del preprocesamiento
df_base = procesar_dataset_aire(
    ruta_archivo="ProyMetodos_DatosCentro.xlsx",
    nombre_salida="datos_interpolados_centro"
)

# Verificación rápida
print(f"Dimensiones del dataset: {df_base.shape}")
df_base.head()

Dataset procesado guardado como: datos_interpolados_centro.csv
Dimensiones del dataset: (18285, 7)


,CO,NO2,PM2.5,VEL,DIR,TMP,RS
Fecha,,,,,,,
2023-10-01 00:00:00,0.611,33.25,9.41,1.11,267.72,13.13,0.0
2023-10-01 01:00:00,0.549,31.67,9.52,1.61,275.16,12.61,0.0
2023-10-01 02:00:00,0.368,20.47,6.12,1.83,274.40,12.06,0.0
2023-10-01 03:00:00,0.286,14.05,3.22,1.95,273.62,11.87,0.0
2023-10-01 04:00:00,0.315,18.22,4.29,2.08,281.30,11.43,0.0


In [11]:
def spline_cubico_natural(x, y, x_eval):
    n = len(x)

    if n < 2:
        return y

    if n == 2:
        return np.interp(x_eval, x, y)

    h = np.diff(x)

    A = np.zeros((n-2, n-2))
    b = np.zeros(n-2)

    for i in range(1, n-1):
        if i-2 >= 0:
            A[i-1, i-2] = h[i-2]
        A[i-1, i-1] = 2 * (h[i-2] + h[i-1])
        if i < n-2:
            A[i-1, i] = h[i-1]

        b[i-1] = 6 * ((y[i+1] - y[i]) / h[i] - (y[i] - y[i-1]) / h[i-1])

    m = np.zeros(n)

    try:
        m[1:n-1] = np.linalg.solve(A, b)
    except:
        m[:] = 0

    def evaluar(z):
        if z <= x[0]:
            return y[0]
        if z >= x[-1]:
            return y[-1]

        i = np.searchsorted(x, z) - 1
        hi = h[i]

        return (
            m[i] * (x[i+1] - z)**3 / (6 * hi)
            + m[i+1] * (z - x[i])**3 / (6 * hi)
            + (y[i] / hi - m[i] * hi / 6) * (x[i+1] - z)
            + (y[i+1] / hi - m[i+1] * hi / 6) * (z - x[i])
        )

    return np.array([evaluar(z) for z in x_eval])


def aplicar_spline(df, variables, hora_inicio=6, hora_fin=18):
    df_out = df.copy()
    x = np.arange(len(df_out))
    horas = df_out.index.hour
    es_dia = (horas >= hora_inicio) & (horas <= hora_fin)

    for var in variables:
        y = df_out[var].values.astype(float)

        if np.isnan(y).sum() > 0:
            continue

        if var == "RS":
            y_suave = y.copy()
            i = 0
            while i < len(y):
                if not es_dia[i]:
                    i += 1
                    continue
                ini = i
                while i < len(y) and es_dia[i]:
                    i += 1
                fin = i
                if fin - ini >= 4:
                    y_suave[ini:fin] = spline_cubico_natural(
                        x[ini:fin], y[ini:fin], x[ini:fin]
                    )
            df_out[var + "_spline"] = y_suave
        else:
            df_out[var + "_spline"] = spline_cubico_natural(x, y, x)

    return df_out